# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [6]:
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    
    int_to_vocab = dict(enumerate(set(text)))
    vocab_to_int = {v:k for k,v in int_to_vocab.items()}
    
    # return tuple
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [9]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation_dict = {'.'  : '||Period||',
                        ','  : '||Comma||',
                        '"'  : '||Quotation_Mark||',
                        ';'  : '||Semicolon||',
                        '!'  : '||Explamation_Mark||',
                        '?'  : '||Question_Mark||',
                        '('  : '||Left_Parenthesis||',
                        ')'  : '||Right_Parenthesis||',
                        '-'  : '||Dash||',
                        '\n' : '||Return||'}

    return punctuation_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [3]:
from torch.utils.data import TensorDataset, DataLoader


def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    
    # NOTE: make sure this function returns integers and not floats. As numbers are used as lookup they need to be interger.
    #       using torch.tensor([...]) turns them into floats!
    
    # create feature tensors 
    feature_tensors = torch.from_numpy(np.array([words[i:i+sequence_length] for i in range(0, len(words)-sequence_length)])) 
    # create target  tensors 
    target_tensors = torch.from_numpy(np.array([words[i+sequence_length] for i in range(0, len(words)-sequence_length)]))
    
    # create tensor dataset and dataloader
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

    # return a dataloader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [4]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[ 18,  19,  20,  21,  22],
        [ 11,  12,  13,  14,  15],
        [ 38,  39,  40,  41,  42],
        [  4,   5,   6,   7,   8],
        [ 24,  25,  26,  27,  28],
        [ 37,  38,  39,  40,  41],
        [ 39,  40,  41,  42,  43],
        [  3,   4,   5,   6,   7],
        [  6,   7,   8,   9,  10],
        [ 34,  35,  36,  37,  38]])

torch.Size([10])
tensor([ 23,  16,  43,   9,  29,  42,  44,   8,  11,  39])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [5]:
import torch.nn as nn
import torch.nn.functional as F

In [26]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        # define model layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(dropout)
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function
        
        # nn_input                                     # batch_size, sequence_length, 1
        
        #print(nn_input.shape)
        batch_size = nn_input.size(0)          
        
        x = self.embedding(nn_input)                   # batch_size, sequence_length, embedding_dim
        #print(x.shape)

        x, hidden = self.lstm(x, hidden)               # batch_size, sequence_length, hidden_dim
        #print(x.shape)
        #x = self.dropout(x)                        
        x = x.contiguous().view(-1, self.hidden_dim)                # batch_size x sequence_length, hidden_dim
        #print(x.shape)
        x = self.fc(x)                                 # batch_size x sequence_length, output_size
        #print(x.shape)
        x = x.view(batch_size, -1, self.output_size)   # batch_size, sequence_length, output_size
        #print(x.shape)
        x = x[:, -1, :]                                # batch_size, output_size (take only the last from the sequence)
        #print(x.shape)

        # return one batch of output word scores and the hidden state
        return x, hidden
    
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [27]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp = inp.cuda()
        target = target.cuda()
        
    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    hidden = tuple([each.data for each in hidden])

    # perform backpropagation and optimization
    optimizer.zero_grad()
    output, hidden = rnn(inp, hidden)
    loss = criterion(output, target)
    loss.backward()
    
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    clip = 5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
        
    optimizer.step()    
    
    # return the loss over a batch and the hidden state produced by our model
    return loss.item(), hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    loss_prev = 3.5
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                
                loss_avg = np.average(batch_losses)
                if loss_avg < 3.5 and loss_avg < loss_prev:
                    print('.. saving model .. ')
                    helper.save_model('./save/trained_rnn_checkpoint', rnn)
                    loss_prev = loss_avg

                batch_losses = []
                
                

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [29]:
# Data params
# Sequence Length
sequence_length = 16  # of words in a sequence
# Batch Size
batch_size = 128

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [30]:
# Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 300
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 2

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [20]:
from workspace_utils import keep_awake, active_session

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
with active_session():
    trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn_v6', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 5.499477212905884
Epoch:    1/15    Loss: 4.862263975143432
Epoch:    1/15    Loss: 4.638069270610809
Epoch:    1/15    Loss: 4.529543158531189
Epoch:    1/15    Loss: 4.424484016418457
Epoch:    1/15    Loss: 4.368171902656555
Epoch:    1/15    Loss: 4.316803344249726
Epoch:    1/15    Loss: 4.281595493793487
Epoch:    1/15    Loss: 4.23137557888031
Epoch:    1/15    Loss: 4.17894561958313
Epoch:    1/15    Loss: 4.184410461425781
Epoch:    1/15    Loss: 4.1551538066864016
Epoch:    1/15    Loss: 4.110859675884247
Epoch:    2/15    Loss: 4.0275660828294155
Epoch:    2/15    Loss: 3.93588555765152
Epoch:    2/15    Loss: 3.9453918414115905
Epoch:    2/15    Loss: 3.9417418885231017
Epoch:    2/15    Loss: 3.919604975223541
Epoch:    2/15    Loss: 3.900815770626068
Epoch:    2/15    Loss: 3.9087136049270628
Epoch:    2/15    Loss: 3.9024212036132813
Epoch:    2/15    Loss: 3.916024523735046
Epoch:    2/15    Loss: 3.9007914566993716
Ep

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch:    5/15    Loss: 3.4764921202659607
Epoch:    5/15    Loss: 3.5019774808883666
Epoch:    5/15    Loss: 3.4962063159942627
Epoch:    5/15    Loss: 3.520909453868866
Epoch:    5/15    Loss: 3.5500350732803345
Epoch:    5/15    Loss: 3.530176666736603
Epoch:    5/15    Loss: 3.538138213634491
Epoch:    5/15    Loss: 3.549851496219635
Epoch:    5/15    Loss: 3.545356610774994
Epoch:    5/15    Loss: 3.549173131465912
Epoch:    5/15    Loss: 3.5681436405181883
Epoch:    6/15    Loss: 3.489511997707119
Epoch:    6/15    Loss: 3.426909979343414
.. saving model .. 
Epoch:    6/15    Loss: 3.407458109855652
.. saving model .. 
Epoch:    6/15    Loss: 3.425656143665314
Epoch:    6/15    Loss: 3.4518227849006653
Epoch:    6/15    Loss: 3.4348945803642272
Epoch:    6/15    Loss: 3.4401622624397277
Epoch:    6/15    Loss: 3.464060878276825
Epoch:    6/15    Loss: 3.4825089054107665
Epoch:    6/15    Loss: 3.4635950207710264
Epoch:    6/15    Loss: 3.4945506176948546
Epoch:    6/15    Loss: 3

### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** (Write answer, here)

I first tried with LR=0.01 as suggested in as the best first choice in the hyperparameter section of this course. The loss decreased a bit but soon started oscillating around one value, so I decided to decrease the learning rate to LR=0.001. For embedding size I chose 300, the hidden dim 256 and number of hidden layers 2. Again, acoording to the hyperparameter section of this course pretty standard choices for a first guess. For batch size I chose 128 to have sufficiently large badges to get some computational advatage, but still have to possibility to introduce some noise between batches. I started out with a sequence length of 32, but decreased this to 16 (and even experimented with 8). I didn't see a big change when changing the sequence length. I suppose a length of 32 is a bit too long, because this is typically longer than a sentence, although it might help to get a more coherent text. A sequence length of 8 seemed to perform a little worse than 16 but differences were marginal.

As I didn't manage to get the loss below 3.5, I experimented with changing the number of layers in the LSTM and changing the number of dimensions of the embedding. All the changes I made only had marginal effects and the loss wouldn't drop below 3.7 (although for some setups the loss was still decreasing after 15 epochs, albeit super slowly).

Then I did some research online and I found that the network architecture I used what slightly different from some proposed solutions: I had a dropout layer between the lstm and the fully connected layer, whereas other solutions had not. So I removed this dropout layer and trained again with the best settings I had found so far, and suddenly my loss dropped very fast.

See the commented cell below for the experiments I ran.

**question: why was it wrong to have a dropout layer between the lstm and the fully connected layer? I am a bit surprised as this dropout layer was used in the examples shown in the course. How can it have such a desastrous effect?**

In [ ]:
# ### version 0
# sequence_length = 32  
# batch_size = 128
# num_epochs = 20
# learning_rate = 0.01
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2

# Training for 20 epoch(s)...
# Epoch:    1/20    Loss: 5.460525995254517
# Epoch:    1/20    Loss: 5.166638576507569
# Epoch:    1/20    Loss: 5.099141284942627
# Epoch:    1/20    Loss: 4.996224870681763
# Epoch:    1/20    Loss: 4.949380994319916
# Epoch:    1/20    Loss: 4.936281491279602
# Epoch:    1/20    Loss: 4.926215611934662
# Epoch:    1/20    Loss: 4.928925151348114
# Epoch:    1/20    Loss: 4.90403492975235
# Epoch:    1/20    Loss: 4.889461074352265
# Epoch:    1/20    Loss: 4.88435894203186

# --> training very slow. Add gradient clipping

# ### version 1
# sequence_length = 32  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.01
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2

# Training for 15 epoch(s)...
# Epoch:    1/15    Loss: 5.499480829238892
# Epoch:    1/15    Loss: 5.19028466796875
# Epoch:    1/15    Loss: 5.128613296508789
# Epoch:    1/15    Loss: 5.086304381370544
# Epoch:    1/15    Loss: 5.038975740432739
# Epoch:    1/15    Loss: 4.970300778388977
# Epoch:    1/15    Loss: 4.9832985982894895
# Epoch:    1/15    Loss: 4.93127595949173
# Epoch:    1/15    Loss: 4.919026745796204
# Epoch:    1/15    Loss: 4.913300184249878
# Epoch:    1/15    Loss: 4.889070321559906
# Epoch:    1/15    Loss: 4.909140929698944
# Epoch:    1/15    Loss: 4.883542202949524
# Epoch:    2/15    Loss: 4.857597909352605
# Epoch:    2/15    Loss: 4.786878496646881
# Epoch:    2/15    Loss: 4.7809897174835205
# Epoch:    2/15    Loss: 4.760116715431213
# Epoch:    2/15    Loss: 4.785763949394226
# Epoch:    2/15    Loss: 4.799982552528381
# Epoch:    2/15    Loss: 4.80301322221756
# Epoch:    2/15    Loss: 4.80504048204422
# Epoch:    2/15    Loss: 4.8079799327850345
# Epoch:    2/15    Loss: 4.793035057067871
# Epoch:    2/15    Loss: 4.783300161361694
# Epoch:    2/15    Loss: 4.806245972156525
# Epoch:    2/15    Loss: 4.788951476573944
# Epoch:    3/15    Loss: 4.758674671283324
# Epoch:    3/15    Loss: 4.728898733615876
# Epoch:    3/15    Loss: 4.766789911270141
# Epoch:    3/15    Loss: 4.7299047346115115
# Epoch:    3/15    Loss: 4.759466106891632

# --> still trains slow, try changing sequence length

# ### version 2
# sequence_length = 16  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.01
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2


# Training for 15 epoch(s)...
# Epoch:    1/15    Loss: 5.455128165245056
# Epoch:    1/15    Loss: 5.212915835380554
# Epoch:    1/15    Loss: 5.114784580230713
# Epoch:    1/15    Loss: 5.032440539360047
# Epoch:    1/15    Loss: 4.973710635185242
# Epoch:    1/15    Loss: 4.970176100730896
# Epoch:    1/15    Loss: 4.95153830242157
# Epoch:    1/15    Loss: 4.901693571567535
# Epoch:    1/15    Loss: 4.925366793632508
# Epoch:    1/15    Loss: 4.891993294715881
# Epoch:    1/15    Loss: 4.881720801353454
# Epoch:    1/15    Loss: 4.9151031374931335
# Epoch:    1/15    Loss: 4.924367157936096
# Epoch:    2/15    Loss: 4.836702711441937
# Epoch:    2/15    Loss: 4.806339129447937
# Epoch:    2/15    Loss: 4.807898818016052

# doesn't look a lot better and loss is oscilation. Try decrease of LR.

# ### version 2 --> name = trained_rnn
# sequence_length = 16  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.001
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2

# Training for 15 epoch(s)...
# Epoch:    1/15    Loss: 5.519032412528992
# Epoch:    1/15    Loss: 4.961615224838257
# Epoch:    1/15    Loss: 4.796547362804413
# Epoch:    1/15    Loss: 4.686773490428925
# Epoch:    1/15    Loss: 4.603153494358063
# Epoch:    1/15    Loss: 4.52300830078125
# Epoch:    1/15    Loss: 4.493691945075989
# Epoch:    1/15    Loss: 4.453806500434876
# Epoch:    1/15    Loss: 4.425888483524322
# Epoch:    1/15    Loss: 4.408744540214538
# Epoch:    1/15    Loss: 4.34470463514328
# Epoch:    1/15    Loss: 4.327123745918274
# Epoch:    1/15    Loss: 4.346908221721649
# Epoch:    2/15    Loss: 4.262341998063866
# Epoch:    2/15    Loss: 4.216996586799621
# Epoch:    2/15    Loss: 4.225688494682312
# Epoch:    2/15    Loss: 4.234294206142425
# Epoch:    2/15    Loss: 4.211466376304626
# Epoch:    2/15    Loss: 4.221382804870606
# Epoch:    2/15    Loss: 4.189474351406098
# Epoch:    2/15    Loss: 4.215312426567078
# Epoch:    2/15    Loss: 4.1884822130203245
# Epoch:    2/15    Loss: 4.183740376472473
# Epoch:    2/15    Loss: 4.173073785305023
# Epoch:    2/15    Loss: 4.181075992107392
# Epoch:    2/15    Loss: 4.157103964805603
# Epoch:    3/15    Loss: 4.132404563104651
# Epoch:    3/15    Loss: 4.096895118713379
# Epoch:    3/15    Loss: 4.055186163425446
# Epoch:    3/15    Loss: 4.075264447212219
# Epoch:    3/15    Loss: 4.07663373708725
# Epoch:    3/15    Loss: 4.07523021364212
# Epoch:    3/15    Loss: 4.079049460887909
# Epoch:    3/15    Loss: 4.081202241420746
# Epoch:    3/15    Loss: 4.098095992565155
# Epoch:    3/15    Loss: 4.086389748573303
# Epoch:    3/15    Loss: 4.095138217926025
# Epoch:    3/15    Loss: 4.097807446956635
# Epoch:    3/15    Loss: 4.087207768917084
# Epoch:    4/15    Loss: 4.02810571658722
# Epoch:    4/15    Loss: 4.007751803398133
# Epoch:    4/15    Loss: 4.004253461360931
# Epoch:    4/15    Loss: 4.015578085422516
# Epoch:    4/15    Loss: 4.005824194908142
# Epoch:    4/15    Loss: 4.031077972888946
# Epoch:    4/15    Loss: 4.002569125652313
# Epoch:    4/15    Loss: 4.025122692584992
# Epoch:    4/15    Loss: 4.033789138793946
# Epoch:    4/15    Loss: 4.000502274036408
# Epoch:    4/15    Loss: 4.015492540836334
# Epoch:    4/15    Loss: 4.029927810192108
# Epoch:    4/15    Loss: 4.026260192394257
# Epoch:    5/15    Loss: 3.9780971415886817
# Epoch:    5/15    Loss: 3.9247016425132752
# Epoch:    5/15    Loss: 3.9394612460136416
# Epoch:    5/15    Loss: 3.9739058747291565
# Epoch:    5/15    Loss: 3.9555334248542784
# Epoch:    5/15    Loss: 3.97061545419693
# Epoch:    5/15    Loss: 3.9877847485542297
# Epoch:    5/15    Loss: 3.9801966304779053
# Epoch:    5/15    Loss: 4.003077658176422
# Epoch:    5/15    Loss: 3.979663116931915
# Epoch:    5/15    Loss: 3.9802927193641664
# Epoch:    5/15    Loss: 3.9686021227836608
# Epoch:    5/15    Loss: 3.98752077627182
# Epoch:    6/15    Loss: 3.942910586348259
# Epoch:    6/15    Loss: 3.8966220421791076
# Epoch:    6/15    Loss: 3.927069453239441
# Epoch:    6/15    Loss: 3.920699739933014
# Epoch:    6/15    Loss: 3.9053760147094727
# Epoch:    6/15    Loss: 3.9325578022003174
# Epoch:    6/15    Loss: 3.962861571311951
# Epoch:    6/15    Loss: 3.936481939315796
# Epoch:    6/15    Loss: 3.9308174114227294
# Epoch:    6/15    Loss: 3.9355156359672545
# Epoch:    6/15    Loss: 3.9285977635383604
# Epoch:    6/15    Loss: 3.950507252216339
# Epoch:    6/15    Loss: 3.967971743583679
# Epoch:    7/15    Loss: 3.8972910488845147
# Epoch:    7/15    Loss: 3.8808460893630983
# Epoch:    7/15    Loss: 3.861058430671692
# Epoch:    7/15    Loss: 3.914267937660217
# Epoch:    7/15    Loss: 3.9037666358947756
# Epoch:    7/15    Loss: 3.9008167457580565
# Epoch:    7/15    Loss: 3.900969645500183
# Epoch:    7/15    Loss: 3.9232785086631776
# Epoch:    7/15    Loss: 3.9365118794441223
# Epoch:    7/15    Loss: 3.9118091592788695
# Epoch:    7/15    Loss: 3.9244997725486757
# Epoch:    7/15    Loss: 3.912750388145447
# Epoch:    7/15    Loss: 3.9259358072280883
# Epoch:    8/15    Loss: 3.8812490186209034
# Epoch:    8/15    Loss: 3.867478670120239
# Epoch:    8/15    Loss: 3.846813494205475
# Epoch:    8/15    Loss: 3.8761486883163454
# Epoch:    8/15    Loss: 3.871365569114685
# Epoch:    8/15    Loss: 3.8890811285972595
# Epoch:    8/15    Loss: 3.8748676557540893
# Epoch:    8/15    Loss: 3.8675901818275453
# Epoch:    8/15    Loss: 3.885761694908142
# Epoch:    8/15    Loss: 3.8838967471122743
# Epoch:    8/15    Loss: 3.8766985912322998
# Epoch:    8/15    Loss: 3.8975093331336974
# Epoch:    8/15    Loss: 3.908369164466858
# Epoch:    9/15    Loss: 3.8871819298825887
# Epoch:    9/15    Loss: 3.844310713291168
# Epoch:    9/15    Loss: 3.814905472755432
# Epoch:    9/15    Loss: 3.8585613536834718
# Epoch:    9/15    Loss: 3.8644566106796265
# Epoch:    9/15    Loss: 3.8708942756652833
# Epoch:    9/15    Loss: 3.87895063829422
# Epoch:    9/15    Loss: 3.8743568601608276
# Epoch:    9/15    Loss: 3.8632457666397095
# Epoch:    9/15    Loss: 3.8567396984100344
# Epoch:    9/15    Loss: 3.8602488312721253
# Epoch:    9/15    Loss: 3.853949611186981
# Epoch:    9/15    Loss: 3.867316078186035
# Epoch:   10/15    Loss: 3.835689097366097
# Epoch:   10/15    Loss: 3.812464605808258
# Epoch:   10/15    Loss: 3.8463937005996702
# Epoch:   10/15    Loss: 3.825820411205292
# Epoch:   10/15    Loss: 3.825621946334839
# Epoch:   10/15    Loss: 3.828979486465454
# Epoch:   10/15    Loss: 3.854597285747528
# Epoch:   10/15    Loss: 3.8347938766479492
# Epoch:   10/15    Loss: 3.824804151058197
# Epoch:   10/15    Loss: 3.885854388713837
# Epoch:   10/15    Loss: 3.843066740989685
# Epoch:   10/15    Loss: 3.878377468585968
# Epoch:   10/15    Loss: 3.8515285577774048
# Epoch:   11/15    Loss: 3.828785253383058
# Epoch:   11/15    Loss: 3.8152266235351564
# Epoch:   11/15    Loss: 3.8097062616348265
# Epoch:   11/15    Loss: 3.8201150331497193
# Epoch:   11/15    Loss: 3.7980953345298767
# Epoch:   11/15    Loss: 3.8118562350273133
# Epoch:   11/15    Loss: 3.8375886673927306
# Epoch:   11/15    Loss: 3.825424122333527
# Epoch:   11/15    Loss: 3.8418018822669984
# Epoch:   11/15    Loss: 3.8220039687156677
# Epoch:   11/15    Loss: 3.844194818019867
# Epoch:   11/15    Loss: 3.8629480285644533
# Epoch:   11/15    Loss: 3.8426440386772156
# Epoch:   12/15    Loss: 3.81013728442945
# Epoch:   12/15    Loss: 3.8092497510910035
# Epoch:   12/15    Loss: 3.7994404263496397
# Epoch:   12/15    Loss: 3.8090615043640135
# Epoch:   12/15    Loss: 3.7880411357879638
# Epoch:   12/15    Loss: 3.8103626985549925
# Epoch:   12/15    Loss: 3.7838774371147155
# Epoch:   12/15    Loss: 3.796249792098999
# Epoch:   12/15    Loss: 3.839239092350006
# Epoch:   12/15    Loss: 3.820247272491455
# Epoch:   12/15    Loss: 3.8409665598869323
# Epoch:   12/15    Loss: 3.835756420135498
# Epoch:   12/15    Loss: 3.8401689910888672
# Epoch:   13/15    Loss: 3.77836883449456
# Epoch:   13/15    Loss: 3.7769511947631838
# Epoch:   13/15    Loss: 3.7653105006217955
# Epoch:   13/15    Loss: 3.7817329602241516
# Epoch:   13/15    Loss: 3.790881278514862
# Epoch:   13/15    Loss: 3.8047273993492126
# Epoch:   13/15    Loss: 3.8052278389930727
# Epoch:   13/15    Loss: 3.801958487510681
# Epoch:   13/15    Loss: 3.7852134947776794
# Epoch:   13/15    Loss: 3.8235168442726137
# Epoch:   13/15    Loss: 3.8050047130584717
# Epoch:   13/15    Loss: 3.809737018585205
# Epoch:   13/15    Loss: 3.8598892188072202
# Epoch:   14/15    Loss: 3.7918350617829
# Epoch:   14/15    Loss: 3.778177189826965
# Epoch:   14/15    Loss: 3.7896382613182067
# Epoch:   14/15    Loss: 3.77360942363739
# Epoch:   14/15    Loss: 3.775358503818512
# Epoch:   14/15    Loss: 3.791509140968323
# Epoch:   14/15    Loss: 3.788397572040558
# Epoch:   14/15    Loss: 3.7955903544425964
# Epoch:   14/15    Loss: 3.785411367893219
# Epoch:   14/15    Loss: 3.7711777596473692
# Epoch:   14/15    Loss: 3.7759347710609434
# Epoch:   14/15    Loss: 3.794540900707245
# Epoch:   14/15    Loss: 3.8259025988578794
# Epoch:   15/15    Loss: 3.7718084138982437
# Epoch:   15/15    Loss: 3.748842754840851
# Epoch:   15/15    Loss: 3.7566493954658506
# Epoch:   15/15    Loss: 3.754053982257843
# Epoch:   15/15    Loss: 3.7736170907020568
# Epoch:   15/15    Loss: 3.7669648213386537
# Epoch:   15/15    Loss: 3.7644480743408204
# Epoch:   15/15    Loss: 3.774913682937622
# Epoch:   15/15    Loss: 3.7927930612564085
# Epoch:   15/15    Loss: 3.7783476905822755
# Epoch:   15/15    Loss: 3.7779460549354553
# Epoch:   15/15    Loss: 3.8305985889434813
# Epoch:   15/15    Loss: 3.8072945370674134


# model is a lot better but loss still doesn't decrease below 3.5. The decrease slowed down a lot and has almost come to
# a holt. Moreover, it seems like loss values are starting to oscillate again. Let's try increasing the number of layers 
# in the lstm.

# ### version 3 
# sequence_length = 16  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.001
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 3

# Training for 15 epoch(s)...
# Epoch:    1/15    Loss: 5.84742033290863
# Epoch:    1/15    Loss: 5.16425643157959
# Epoch:    1/15    Loss: 4.934733777523041
# Epoch:    1/15    Loss: 4.853495078086853
# Epoch:    1/15    Loss: 4.762739520072937
# Epoch:    1/15    Loss: 4.733124754905701
# Epoch:    1/15    Loss: 4.635533615589142
# Epoch:    1/15    Loss: 4.633195798397065
# Epoch:    1/15    Loss: 4.556114752292633
# Epoch:    1/15    Loss: 4.540090408802032
# Epoch:    1/15    Loss: 4.5089478025436405
# Epoch:    1/15    Loss: 4.4759446077346805
# Epoch:    1/15    Loss: 4.464113655567169
# Epoch:    2/15    Loss: 4.400235572712586
# Epoch:    2/15    Loss: 4.349160491943359
# Epoch:    2/15    Loss: 4.33269228887558
# Epoch:    2/15    Loss: 4.331838968276977
# Epoch:    2/15    Loss: 4.306009502887726
# Epoch:    2/15    Loss: 4.315991315841675
# Epoch:    2/15    Loss: 4.301412644386292
# Epoch:    2/15    Loss: 4.307970724105835
# Epoch:    2/15    Loss: 4.282957056045532
# Epoch:    2/15    Loss: 4.289683323860168
# Epoch:    2/15    Loss: 4.2938444371223445
# Epoch:    2/15    Loss: 4.270395268440247
# Epoch:    2/15    Loss: 4.261836561203003
# Epoch:    3/15    Loss: 4.218111871073735
# Epoch:    3/15    Loss: 4.212543261528015
# Epoch:    3/15    Loss: 4.192621736049652
# Epoch:    3/15    Loss: 4.15851506280899
# Epoch:    3/15    Loss: 4.177847417831421
# Epoch:    3/15    Loss: 4.173798290252686
# Epoch:    3/15    Loss: 4.186339723587036
# Epoch:    3/15    Loss: 4.1676453580856325
# Epoch:    3/15    Loss: 4.1744033155441285
# Epoch:    3/15    Loss: 4.167974296569824
# Epoch:    3/15    Loss: 4.163632153511047
# Epoch:    3/15    Loss: 4.185546479225159
# Epoch:    3/15    Loss: 4.1280517992973325
# Epoch:    4/15    Loss: 4.12123004939903
# Epoch:    4/15    Loss: 4.069141949176788
# Epoch:    4/15    Loss: 4.078976782798767
# Epoch:    4/15    Loss: 4.104117788314819
# Epoch:    4/15    Loss: 4.093045629501343
# Epoch:    4/15    Loss: 4.1025740704536435
# Epoch:    4/15    Loss: 4.110804492473602
# Epoch:    4/15    Loss: 4.104619957447052
# Epoch:    4/15    Loss: 4.099385446548462
# Epoch:    4/15    Loss: 4.106912734031678
# Epoch:    4/15    Loss: 4.086147955417633
# Epoch:    4/15    Loss: 4.1034552493095395
# Epoch:    4/15    Loss: 4.081041200637817
# Epoch:    5/15    Loss: 4.076099398704505
# Epoch:    5/15    Loss: 4.035792200565338
# Epoch:    5/15    Loss: 4.029300180912018
# Epoch:    5/15    Loss: 4.044972764492035
# Epoch:    5/15    Loss: 4.040993339538574
# Epoch:    5/15    Loss: 4.031752397537232
# Epoch:    5/15    Loss: 4.0272267394065855
# Epoch:    5/15    Loss: 4.053493663787842
# Epoch:    5/15    Loss: 4.062334110736847
# Epoch:    5/15    Loss: 4.047339728832245
# Epoch:    5/15    Loss: 4.032673896312714
# Epoch:    5/15    Loss: 4.056191856861115
# Epoch:    5/15    Loss: 4.048480787277222
# Epoch:    6/15    Loss: 4.011361232359958
# Epoch:    6/15    Loss: 3.9871636972427367
# Epoch:    6/15    Loss: 4.004178217411042
# Epoch:    6/15    Loss: 3.984003448486328
# Epoch:    6/15    Loss: 3.9939717955589296
# Epoch:    6/15    Loss: 3.9745011796951295
# Epoch:    6/15    Loss: 4.008615599632263
# Epoch:    6/15    Loss: 4.020281545639038
# Epoch:    6/15    Loss: 4.002414772033691
# Epoch:    6/15    Loss: 4.007482011795044
# Epoch:    6/15    Loss: 3.9993919320106506
# Epoch:    6/15    Loss: 4.027185010910034
# Epoch:    6/15    Loss: 4.015576354026795
# Epoch:    7/15    Loss: 3.9869392803947994
# Epoch:    7/15    Loss: 3.955290759563446
# Epoch:    7/15    Loss: 3.956685152053833
# Epoch:    7/15    Loss: 3.9622031569480898
# Epoch:    7/15    Loss: 3.967267558097839

# training is slower than version 2. I guess this makes sense because there are more parameters. Loss is still decreasing
# so with many epochs it might decrease under 3.5, but I do not have the impression it is much better than model version 1,
# so I might as well train that one further. For the next tryout I set the number of layers back to 2 and decrease the 
# sequence length a bit more.

# ### version 4 --> name = trained_rnn_v4
# sequence_length = 8  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.001
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2

# Epoch:    1/15    Loss: 5.518049524307251
# Epoch:    1/15    Loss: 4.940155109405517
# Epoch:    1/15    Loss: 4.777937488079071
# Epoch:    1/15    Loss: 4.673612776756286
# Epoch:    1/15    Loss: 4.58071036529541
# Epoch:    1/15    Loss: 4.523212635517121
# Epoch:    1/15    Loss: 4.47628528881073
# Epoch:    1/15    Loss: 4.448713034152985
# Epoch:    1/15    Loss: 4.424231845378876
# Epoch:    1/15    Loss: 4.4077493114471435
# Epoch:    1/15    Loss: 4.37193318605423
# Epoch:    1/15    Loss: 4.37202109003067
# Epoch:    1/15    Loss: 4.35201669883728
# Epoch:    2/15    Loss: 4.287079760164668
# Epoch:    2/15    Loss: 4.237461950778961
# Epoch:    2/15    Loss: 4.210178918838501
# Epoch:    2/15    Loss: 4.219593146324158
# Epoch:    2/15    Loss: 4.212755613803863
# Epoch:    2/15    Loss: 4.227347667217255
# Epoch:    2/15    Loss: 4.225680639266968
# Epoch:    2/15    Loss: 4.210152006626129
# Epoch:    2/15    Loss: 4.186505747318268
# Epoch:    2/15    Loss: 4.187503507137299
# Epoch:    2/15    Loss: 4.179704353809357
# Epoch:    2/15    Loss: 4.164333332538605
# Epoch:    2/15    Loss: 4.188432929515838
# Epoch:    3/15    Loss: 4.124833865308418
# Epoch:    3/15    Loss: 4.100823547363281
# Epoch:    3/15    Loss: 4.0866428546905516
# Epoch:    3/15    Loss: 4.0944997916221615
# Epoch:    3/15    Loss: 4.116708047866822
# Epoch:    3/15    Loss: 4.068793312549591
# Epoch:    3/15    Loss: 4.089380644798279
# Epoch:    3/15    Loss: 4.09476297044754
# Epoch:    3/15    Loss: 4.119767175674438
# Epoch:    3/15    Loss: 4.071884889602661
# Epoch:    3/15    Loss: 4.101938007831573
# Epoch:    3/15    Loss: 4.07889670753479
# Epoch:    3/15    Loss: 4.134579343318939
# Epoch:    4/15    Loss: 4.048703473664904
# Epoch:    4/15    Loss: 4.019881781578064
# Epoch:    4/15    Loss: 4.002887194633484
# Epoch:    4/15    Loss: 4.022186814308166
# Epoch:    4/15    Loss: 4.030150586128235
# Epoch:    4/15    Loss: 4.0010599312782285
# Epoch:    4/15    Loss: 4.0164085593223575
# Epoch:    4/15    Loss: 4.030945345401764
# Epoch:    4/15    Loss: 4.026732368469238
# Epoch:    4/15    Loss: 4.058487289428711
# Epoch:    4/15    Loss: 4.023422266483307
# Epoch:    4/15    Loss: 4.04203138256073
# Epoch:    4/15    Loss: 4.041637385845184
# Epoch:    5/15    Loss: 3.995131918028289
# Epoch:    5/15    Loss: 3.9613971037864686
# Epoch:    5/15    Loss: 3.950795984745026
# Epoch:    5/15    Loss: 3.965422715187073
# Epoch:    5/15    Loss: 3.9580140480995176
# Epoch:    5/15    Loss: 3.9832658596038817
# Epoch:    5/15    Loss: 3.9760872592926026
# Epoch:    5/15    Loss: 4.002781844139099
# Epoch:    5/15    Loss: 4.009673274993896
# Epoch:    5/15    Loss: 3.9810289697647097
# Epoch:    5/15    Loss: 4.014034676551819
# Epoch:    5/15    Loss: 4.035238174915314
# Epoch:    5/15    Loss: 4.023027569293975
# Epoch:    6/15    Loss: 3.9491277986020616
# Epoch:    6/15    Loss: 3.940419569015503
# Epoch:    6/15    Loss: 3.9507215576171877
# Epoch:    6/15    Loss: 3.939546772003174
# Epoch:    6/15    Loss: 3.9426710567474363
# Epoch:    6/15    Loss: 3.938462323188782
# Epoch:    6/15    Loss: 3.937221574783325
# Epoch:    6/15    Loss: 3.966616075515747
# Epoch:    6/15    Loss: 3.9532057123184203
# Epoch:    6/15    Loss: 3.933286354064941
# Epoch:    6/15    Loss: 3.969691168308258
# Epoch:    6/15    Loss: 3.9676631565093996
# Epoch:    6/15    Loss: 3.960980284690857
# Epoch:    7/15    Loss: 3.9294715154035664
# Epoch:    7/15    Loss: 3.9026622681617735
# Epoch:    7/15    Loss: 3.9025230469703676
# Epoch:    7/15    Loss: 3.9181969041824343
# Epoch:    7/15    Loss: 3.921288127422333
# Epoch:    7/15    Loss: 3.907371584415436
# Epoch:    7/15    Loss: 3.911840890407562
# Epoch:    7/15    Loss: 3.9092805252075196
# Epoch:    7/15    Loss: 3.9380512890815735
# Epoch:    7/15    Loss: 3.942014253616333
# Epoch:    7/15    Loss: 3.915926339626312
# Epoch:    7/15    Loss: 3.938901370048523
# Epoch:    7/15    Loss: 3.945243862628937
# Epoch:    8/15    Loss: 3.9023090095342865
# Epoch:    8/15    Loss: 3.908174484729767
# Epoch:    8/15    Loss: 3.879822171211243
# Epoch:    8/15    Loss: 3.8873110175132752
# Epoch:    8/15    Loss: 3.8891163506507875
# Epoch:    8/15    Loss: 3.895305103302002
# Epoch:    8/15    Loss: 3.913456187725067
# Epoch:    8/15    Loss: 3.918669684410095
# Epoch:    8/15    Loss: 3.88721671628952
# Epoch:    8/15    Loss: 3.9013718066215515
# Epoch:    8/15    Loss: 3.898134192943573
# Epoch:    8/15    Loss: 3.9231419515609742
# Epoch:    8/15    Loss: 3.91435280752182
# Epoch:    9/15    Loss: 3.895524090903708
# Epoch:    9/15    Loss: 3.871891031742096
# Epoch:    9/15    Loss: 3.8609390077590944
# Epoch:    9/15    Loss: 3.849773305892944
# Epoch:    9/15    Loss: 3.848509888648987
# Epoch:    9/15    Loss: 3.878687014102936
# Epoch:    9/15    Loss: 3.8866572360992433
# Epoch:    9/15    Loss: 3.8901185822486877
# Epoch:    9/15    Loss: 3.9035669364929197
# Epoch:    9/15    Loss: 3.8850576882362367
# Epoch:    9/15    Loss: 3.8888314533233643
# Epoch:    9/15    Loss: 3.90793292760849
# Epoch:    9/15    Loss: 3.8963913893699647
# Epoch:   10/15    Loss: 3.8565120820044485
# Epoch:   10/15    Loss: 3.8460610241889954
# Epoch:   10/15    Loss: 3.825290771961212
# Epoch:   10/15    Loss: 3.8533532614707946
# Epoch:   10/15    Loss: 3.855859495639801
# Epoch:   10/15    Loss: 3.8580910811424256
# Epoch:   10/15    Loss: 3.868948299884796
# Epoch:   10/15    Loss: 3.8613072752952577
# Epoch:   10/15    Loss: 3.8646263666152953
# Epoch:   10/15    Loss: 3.8805489711761476
# Epoch:   10/15    Loss: 3.8742265372276306
# Epoch:   10/15    Loss: 3.8958911356925965
# Epoch:   10/15    Loss: 3.873976909637451
# Epoch:   11/15    Loss: 3.8366952911865106
# Epoch:   11/15    Loss: 3.804987115383148
# Epoch:   11/15    Loss: 3.83410249376297
# Epoch:   11/15    Loss: 3.8454462718963622
# Epoch:   11/15    Loss: 3.8420352325439455
# Epoch:   11/15    Loss: 3.8496437230110168
# Epoch:   11/15    Loss: 3.849741261959076
# Epoch:   11/15    Loss: 3.8248952951431274
# Epoch:   11/15    Loss: 3.851562162399292
# Epoch:   11/15    Loss: 3.854656102657318
# Epoch:   11/15    Loss: 3.8774023966789244
# Epoch:   11/15    Loss: 3.843923249721527
# Epoch:   11/15    Loss: 3.8682328658103944
# Epoch:   12/15    Loss: 3.8363636447918306
# Epoch:   12/15    Loss: 3.821626567363739
# Epoch:   12/15    Loss: 3.8248793678283692
# Epoch:   12/15    Loss: 3.819521810054779
# Epoch:   12/15    Loss: 3.810843094825745
# Epoch:   12/15    Loss: 3.814983813285828
# Epoch:   12/15    Loss: 3.8416836376190187
# Epoch:   12/15    Loss: 3.811550538063049
# Epoch:   12/15    Loss: 3.8471876101493834
# Epoch:   12/15    Loss: 3.8451995968818666
# Epoch:   12/15    Loss: 3.8380072112083434
# Epoch:   12/15    Loss: 3.8582865862846374
# Epoch:   12/15    Loss: 3.8387609195709227
# Epoch:   13/15    Loss: 3.8016239332094774
# Epoch:   13/15    Loss: 3.8020752811431886
# Epoch:   13/15    Loss: 3.816856801509857
# Epoch:   13/15    Loss: 3.7832143297195433
# Epoch:   13/15    Loss: 3.8105930733680724
# Epoch:   13/15    Loss: 3.8188009767532347
# Epoch:   13/15    Loss: 3.8213028264045716
# Epoch:   13/15    Loss: 3.82004314994812
# Epoch:   13/15    Loss: 3.8113063125610354
# Epoch:   13/15    Loss: 3.8313876605033874
# Epoch:   13/15    Loss: 3.838060401916504
# Epoch:   13/15    Loss: 3.8405387077331543
# Epoch:   13/15    Loss: 3.832410590171814
# Epoch:   14/15    Loss: 3.797427784916786
# Epoch:   14/15    Loss: 3.7786704711914063
# Epoch:   14/15    Loss: 3.776788297176361
# Epoch:   14/15    Loss: 3.8004076132774354
# Epoch:   14/15    Loss: 3.7763463439941405
# Epoch:   14/15    Loss: 3.8003002076148986
# Epoch:   14/15    Loss: 3.7921651248931885
# Epoch:   14/15    Loss: 3.8289710249900817
# Epoch:   14/15    Loss: 3.8254407625198366
# Epoch:   14/15    Loss: 3.813517626285553
# Epoch:   14/15    Loss: 3.8200611534118654
# Epoch:   14/15    Loss: 3.8512700262069703
# Epoch:   14/15    Loss: 3.8459946026802063
# Epoch:   15/15    Loss: 3.805055347389481
# Epoch:   15/15    Loss: 3.7899530997276307
# Epoch:   15/15    Loss: 3.7821882696151734
# Epoch:   15/15    Loss: 3.7852020678520204
# Epoch:   15/15    Loss: 3.779290771007538
# Epoch:   15/15    Loss: 3.7802586250305175
# Epoch:   15/15    Loss: 3.785729476928711
# Epoch:   15/15    Loss: 3.7948217883110047
# Epoch:   15/15    Loss: 3.800333023071289
# Epoch:   15/15    Loss: 3.790686616420746
# Epoch:   15/15    Loss: 3.837830164909363
# Epoch:   15/15    Loss: 3.816711311817169
# Epoch:   15/15    Loss: 3.8311368837356565


# training slightly slower than version 2, and model seems not to improve after epoch 13. Did some googling
# and found a solution where loss decreases below 3.5 after 5 epochs. The main difference in hyperparameters seems to be the
# embedding dim. (strange observation: no dropout used between lstm and fc. Maybe this also has an effect? --> version 6) Try decreasing the embedding dim to 200


# ### version 5 --> name = trained_rnn_v5
# sequence_length = 8  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.001
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 200
# hidden_dim = 256
# n_layers = 2


# Epoch:    1/15    Loss: 5.593783767700195
# Epoch:    1/15    Loss: 5.033397290229797
# Epoch:    1/15    Loss: 4.867132264137268
# Epoch:    1/15    Loss: 4.736566205978393
# Epoch:    1/15    Loss: 4.679233968734741
# Epoch:    1/15    Loss: 4.589180280208588
# Epoch:    1/15    Loss: 4.522088750362396
# Epoch:    1/15    Loss: 4.522383358478546
# Epoch:    1/15    Loss: 4.456399173736572
# Epoch:    1/15    Loss: 4.433870898723602
# Epoch:    1/15    Loss: 4.465030921936036
# Epoch:    1/15    Loss: 4.408910404205322
# Epoch:    1/15    Loss: 4.3726293601989745
# Epoch:    2/15    Loss: 4.322213701419417
# Epoch:    2/15    Loss: 4.268416977405548
# Epoch:    2/15    Loss: 4.260474427700043
# Epoch:    2/15    Loss: 4.242636108398438
# Epoch:    2/15    Loss: 4.240808621883392
# Epoch:    2/15    Loss: 4.254262212753296
# Epoch:    2/15    Loss: 4.233557104587555
# Epoch:    2/15    Loss: 4.228321288585663
# Epoch:    2/15    Loss: 4.216945755958557
# Epoch:    2/15    Loss: 4.213841989040374
# Epoch:    2/15    Loss: 4.216994740486145
# Epoch:    2/15    Loss: 4.2124588866233825
# Epoch:    2/15    Loss: 4.211891679286957
# Epoch:    3/15    Loss: 4.178246349742169
# Epoch:    3/15    Loss: 4.115081734180451
# Epoch:    3/15    Loss: 4.123327114582062
# Epoch:    3/15    Loss: 4.13861212682724
# Epoch:    3/15    Loss: 4.115945699214936
# Epoch:    3/15    Loss: 4.121018237113953
# Epoch:    3/15    Loss: 4.132500154018402
# Epoch:    3/15    Loss: 4.1219871234893795
# Epoch:    3/15    Loss: 4.101944729804993
# Epoch:    3/15    Loss: 4.103368011474609
# Epoch:    3/15    Loss: 4.115984516620636
# Epoch:    3/15    Loss: 4.10236794090271
# Epoch:    3/15    Loss: 4.092312212467194
# Epoch:    4/15    Loss: 4.0633727446183086
# Epoch:    4/15    Loss: 4.029348337650299
# Epoch:    4/15    Loss: 4.020329390048981
# Epoch:    4/15    Loss: 4.0462005639076235
# Epoch:    4/15    Loss: 4.050979965686798
# Epoch:    4/15    Loss: 4.0421453304290775
# Epoch:    4/15    Loss: 4.030623759269714
# Epoch:    4/15    Loss: 4.025857295036316
# Epoch:    4/15    Loss: 4.071916033267975
# Epoch:    4/15    Loss: 4.049892538547516
# Epoch:    4/15    Loss: 4.048440396785736
# Epoch:    4/15    Loss: 4.058176394462586
# Epoch:    4/15    Loss: 4.054175399303436
# Epoch:    5/15    Loss: 4.019670555835169
# Epoch:    5/15    Loss: 3.9869855580329894
# Epoch:    5/15    Loss: 3.977295287132263
# Epoch:    5/15    Loss: 3.989190298080444
# Epoch:    5/15    Loss: 3.99208543920517
# Epoch:    5/15    Loss: 3.975911647796631
# Epoch:    5/15    Loss: 4.000957586765289
# Epoch:    5/15    Loss: 4.006728607654572
# Epoch:    5/15    Loss: 4.022181975841522
# Epoch:    5/15    Loss: 3.996363996505737
# Epoch:    5/15    Loss: 3.997881300449371
# Epoch:    5/15    Loss: 3.9982907605171203
# Epoch:    5/15    Loss: 4.020082766532898
# Epoch:    6/15    Loss: 3.9732370612791077
# Epoch:    6/15    Loss: 3.9484042687416077
# Epoch:    6/15    Loss: 3.940296534061432
# Epoch:    6/15    Loss: 3.9390790586471556
# Epoch:    6/15    Loss: 3.957584948539734
# Epoch:    6/15    Loss: 3.947989649772644
# Epoch:    6/15    Loss: 3.963426935195923
# Epoch:    6/15    Loss: 3.9496360397338868
# Epoch:    6/15    Loss: 3.9618434944152834
# Epoch:    6/15    Loss: 3.9667569799423217
# Epoch:    6/15    Loss: 3.9579339814186096
# Epoch:    6/15    Loss: 4.004655180931091
# Epoch:    6/15    Loss: 3.9784385199546812
# Epoch:    7/15    Loss: 3.927567584595813
# Epoch:    7/15    Loss: 3.9039466805458067
# Epoch:    7/15    Loss: 3.9308604316711424
# Epoch:    7/15    Loss: 3.917661540985107
# Epoch:    7/15    Loss: 3.907239541053772
# Epoch:    7/15    Loss: 3.9486536068916323
# Epoch:    7/15    Loss: 3.9315218386650086
# Epoch:    7/15    Loss: 3.9352730598449708
# Epoch:    7/15    Loss: 3.9445277976989748
# Epoch:    7/15    Loss: 3.933239758491516
# Epoch:    7/15    Loss: 3.9339395856857298
# Epoch:    7/15    Loss: 3.9558816471099854
# Epoch:    7/15    Loss: 3.924215931415558
# Epoch:    8/15    Loss: 3.8901916682043556
# Epoch:    8/15    Loss: 3.8947464289665223
# Epoch:    8/15    Loss: 3.8821212124824522
# Epoch:    8/15    Loss: 3.8865860080718995
# Epoch:    8/15    Loss: 3.9041392822265624
# Epoch:    8/15    Loss: 3.887415068626404
# Epoch:    8/15    Loss: 3.8985217623710633
# Epoch:    8/15    Loss: 3.9146260714530943
# Epoch:    8/15    Loss: 3.9246347131729125
# Epoch:    8/15    Loss: 3.9062631740570066
# Epoch:    8/15    Loss: 3.896082332611084
# Epoch:    8/15    Loss: 3.917540820121765
# Epoch:    8/15    Loss: 3.9310172104835512
# Epoch:    9/15    Loss: 3.8773280201312557
# Epoch:    9/15    Loss: 3.8702488045692442
# Epoch:    9/15    Loss: 3.8786174073219297
# Epoch:    9/15    Loss: 3.8895481662750244
# Epoch:    9/15    Loss: 3.8707076683044432
# Epoch:    9/15    Loss: 3.8561072487831116
# Epoch:    9/15    Loss: 3.9068599395751953
# Epoch:    9/15    Loss: 3.8873358755111695
# Epoch:    9/15    Loss: 3.8716905646324156
# Epoch:    9/15    Loss: 3.9124685368537904
# Epoch:    9/15    Loss: 3.8751187796592714
# Epoch:    9/15    Loss: 3.886350929737091
# Epoch:    9/15    Loss: 3.8873072996139526
# Epoch:   10/15    Loss: 3.851796323915999
# Epoch:   10/15    Loss: 3.8309527139663695
# Epoch:   10/15    Loss: 3.837389994621277
# Epoch:   10/15    Loss: 3.84459161901474
# Epoch:   10/15    Loss: 3.86444873380661
# Epoch:   10/15    Loss: 3.8547330131530764
# Epoch:   10/15    Loss: 3.855164559841156
# Epoch:   10/15    Loss: 3.8802364044189455
# Epoch:   10/15    Loss: 3.8899385390281678
# Epoch:   10/15    Loss: 3.8697053594589232
# Epoch:   10/15    Loss: 3.865320349216461
# Epoch:   10/15    Loss: 3.864398619174957
# Epoch:   10/15    Loss: 3.8897892355918886
# Epoch:   11/15    Loss: 3.8562705433159543
# Epoch:   11/15    Loss: 3.7892336802482607
# Epoch:   11/15    Loss: 3.8272992296218873
# Epoch:   11/15    Loss: 3.8325835852622987
# Epoch:   11/15    Loss: 3.837945197105408
# Epoch:   11/15    Loss: 3.831588062763214
# Epoch:   11/15    Loss: 3.8549895462989805
# Epoch:   11/15    Loss: 3.8522996034622192
# Epoch:   11/15    Loss: 3.856383089065552
# Epoch:   11/15    Loss: 3.8574626927375792
# Epoch:   11/15    Loss: 3.8424164915084837
# Epoch:   11/15    Loss: 3.8668625855445864
# Epoch:   11/15    Loss: 3.8640488028526305
# Epoch:   12/15    Loss: 3.833970496036935
# Epoch:   12/15    Loss: 3.805811495304108
# Epoch:   12/15    Loss: 3.811157021522522
# Epoch:   12/15    Loss: 3.8219465827941894
# Epoch:   12/15    Loss: 3.8391111578941346
# Epoch:   12/15    Loss: 3.8082381925582887
# Epoch:   12/15    Loss: 3.859155685424805
# Epoch:   12/15    Loss: 3.8383434557914735
# Epoch:   12/15    Loss: 3.8177978706359865
# Epoch:   12/15    Loss: 3.8527627563476563
# Epoch:   12/15    Loss: 3.8424286251068116
# Epoch:   12/15    Loss: 3.863317889690399
# Epoch:   12/15    Loss: 3.8266893634796144
# Epoch:   13/15    Loss: 3.8104012708791886
# Epoch:   13/15    Loss: 3.7837746372222902
# Epoch:   13/15    Loss: 3.772832250595093
# Epoch:   13/15    Loss: 3.8040843663215638
# Epoch:   13/15    Loss: 3.8080928077697753
# Epoch:   13/15    Loss: 3.8427132387161254
# Epoch:   13/15    Loss: 3.8235775690078735
# Epoch:   13/15    Loss: 3.8085708322525025
# Epoch:   13/15    Loss: 3.8202431054115293
# Epoch:   13/15    Loss: 3.824189853668213
# Epoch:   13/15    Loss: 3.824763933181763
# Epoch:   13/15    Loss: 3.841682536125183
# Epoch:   13/15    Loss: 3.859227063179016
# Epoch:   14/15    Loss: 3.8040416265296737
# Epoch:   14/15    Loss: 3.784176441192627
# Epoch:   14/15    Loss: 3.7590295386314394
# Epoch:   14/15    Loss: 3.787293918609619
# Epoch:   14/15    Loss: 3.7887982301712038
# Epoch:   14/15    Loss: 3.794253103733063
# Epoch:   14/15    Loss: 3.7983375911712645
# Epoch:   14/15    Loss: 3.7984695677757263
# Epoch:   14/15    Loss: 3.821404613494873
# Epoch:   14/15    Loss: 3.8044066643714904
# Epoch:   14/15    Loss: 3.8281376843452453
# Epoch:   14/15    Loss: 3.8123663144111632
# Epoch:   14/15    Loss: 3.8403959646224974
# Epoch:   15/15    Loss: 3.8115680830397474
# Epoch:   15/15    Loss: 3.7829807863235474
# Epoch:   15/15    Loss: 3.784916895866394
# Epoch:   15/15    Loss: 3.7853982138633726
# Epoch:   15/15    Loss: 3.767937439918518
# Epoch:   15/15    Loss: 3.7672590441703795
# Epoch:   15/15    Loss: 3.805336149215698
# Epoch:   15/15    Loss: 3.7928282170295717
# Epoch:   15/15    Loss: 3.780804820537567
# Epoch:   15/15    Loss: 3.8144318165779114
# Epoch:   15/15    Loss: 3.8214995107650758
# Epoch:   15/15    Loss: 3.7965988717079164
# Epoch:   15/15    Loss: 3.818610242843628

# embedding of 200 compared to 300 does not improve the results. Next, I change the parameters back to those of version 2, and
# remove the dropout between lstm and fc.

# ### version 6 --> name = trained_rnn_v6
# sequence_length = 16  
# batch_size = 128
# num_epochs = 15
# learning_rate = 0.001
# vocab_size = len(int_to_vocab)
# output_size = vocab_size
# embedding_dim = 300
# hidden_dim = 256
# n_layers = 2

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn_v6')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [34]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'jerry' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

jerry: a couple of women have thousands- thirty.

george:(pointing) oh, yeah!(he jumps up and heads into the dressing room)

elaine: oh, hi.

elaine: hey, how you doing?

george: what?

jerry: i don't think so.

george: i mean it's a good idea.

george:(to jerry) oh, hi, uh, i got a big problem in the morning.

kramer: yeah, but it's a good thing, it's a big deal.

jerry: oh, you think i can attract rarely to be?

george: i don't know. i know.

elaine: well i think i can go..

jerry: i don't know.

george: oh, i don't think so.

george: oh, no. no, no. no, no, no, no, no. no. no. no, no, no, no, no, no, i don't want you to do this.

jerry: oh, i don't think so.

george: i know, i don't know. i know what i was thinking about, but i know, i think i may have been a little nervous.

jerry: i don't know what to do. you got a little problem with you.

george: i think i could get out of this place.(to kramer) i was wondering, i don't think so.

elaine:(to george) hey jerry, i got to get some 

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [35]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.